In [7]:
import os
from openai import AsyncOpenAI
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat

from semantic_kernel.functions import KernelFunctionFromPrompt
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.contents import AuthorRole, ChatMessageContent

Create client, add service client to kernel

In [8]:
client = AsyncOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    base_url=f"{os.environ["AZURE_OPENAI_ENDPOINT"]}/openai/deployments/{os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]}",
    default_headers={"api-key": os.environ["AZURE_OPENAI_API_KEY"]},
    default_query={"api-version": os.environ.get("AZURE_OPENAI_API_VERSION", "2025-01-01-preview")},
)
client_service = OpenAIChatCompletion(
    ai_model_id=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    async_client=client
)
kernel = Kernel()
kernel.add_service(client_service)

Create agents

In [9]:
REVIEWER_NAME = "Concierge"
REVIEWER_INSTRUCTIONS = """
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
"""
FRONTDESK_NAME = "FrontDesk"
FRONTDESK_INSTRUCTIONS = """
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
"""
agent_reviewer = ChatCompletionAgent(
    kernel=kernel,
    name=REVIEWER_NAME,
    instructions=REVIEWER_INSTRUCTIONS,
)
agent_writer = ChatCompletionAgent(
    kernel=kernel,
    name=FRONTDESK_NAME,
    instructions=FRONTDESK_INSTRUCTIONS,
)

Write tool use prompt

In [10]:
# Determine when the conversation should end
termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
    Determine if the recommendation process is complete.
    The process is complete when the Concierge provides approval for any recommendation made by the Front Desk.
    Look for phrases like "approved", "this recommendation is approved", or any clear indication that the Concierge is satisfied with the suggestion.
    If the Concierge has given approval in their most recent response, respond with: yes
    Otherwise, respond with: no
    History:
    {{$history}}
    """,
)
# Determine which agent should speak next in the rotation of the conversation.
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    Determine which participant takes the next turn in a conversation based on the the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.
    
    Choose only from these participants:
    - {REVIEWER_NAME}
    - {FRONTDESK_NAME}
    
    Always follow these rules when selecting the next participant, each conversation should be at least 4 turns:
    - After user input, it is {FRONTDESK_NAME}'s turn.
    - After {FRONTDESK_NAME} replies, it is {REVIEWER_NAME}'s turn.
    - After {REVIEWER_NAME} provides feedback, it is {FRONTDESK_NAME}'s turn.
    History:
    {{{{$history}}}}
    """,
)

Create group chat

In [11]:
chat = AgentGroupChat(
    agents=[agent_reviewer, agent_writer],
    #Stop if the reviewer says 'approved' (as prompted)
    termination_strategy=KernelFunctionTerminationStrategy(
        maximum_iterations=10,
        agents=[agent_reviewer],
        history_variable_name="history", # Assign data thread to $history variable in prompt
        kernel=kernel,
        function=termination_function,
        result_parser=lambda result: str(result.value[0]).lower() == "yes",
    ),
    # Select the agent to speak next 
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: str(
            result.value[0]) if result.value is not None else FRONTDESK_NAME,
        agent_variable_name="agents",
        history_variable_name="history",
    ),
)
user_input = "I would like to go to Paris."
chat_mess = ChatMessageContent(role=AuthorRole.USER, content=user_input)

await chat.add_chat_message(chat_mess) # Add to chat history
print(f"# User: '{user_input}'")

async for content in chat.invoke(): # Call chat.invoke() to trigger the conversation flow between agents in AgentGroupChat
    print(f"# Agent - {content.name or '*'}: '{content.content}'")
print(f"# IS COMPLETE: {chat.is_complete}")

# User: 'I would like to go to Paris.'
# Agent - FrontDesk: 'Visit the Eiffel Tower for stunning views of the city.'
# Agent - Concierge: 'While the Eiffel Tower is an iconic landmark, it's also one of the most tourist-heavy spots in Paris. To refine the recommendation, consider suggesting experiences that immerse the traveler in the local culture, such as visiting lesser-known neighborhoods, local markets, or engaging in community events. This will provide a deeper and more authentic experience of Paris.'
# Agent - FrontDesk: 'Explore the Montmartre district for a charming village atmosphere and stunning art scene.'
# Agent - Concierge: 'While Montmartre has its charm, it can still attract a good number of tourists. To enhance the recommendation, consider suggesting a visit to a lesser-known neighborhood or an artist's studio, where travelers might have the chance to interact with locals or partake in a creative workshop. This will ensure a more unique and authentic experience in Pari